In [1]:
import sys
import os
import cv2
import glob
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
import pylidc as pl
from tqdm import tqdm, trange
from configparser import ConfigParser
from statistics import median_high
from utils import (
    is_dir_path,
    segment_lung,
    ct_img_preprocess,
    mask_find_bboxs,
    convert_bbox_to_yolo,
    yolo_bbox_to_str,
    normalize_img,
)
from pylidc.utils import consensus
from PIL import Image
from sklearn.model_selection import train_test_split
import scipy
import vtk

In [2]:
class Nodule(object):
    def __init__(self, patient_id, nodule_no, slice_thickness):
        # 患者id+结节id 是唯一id
        self.patient_id = patient_id # 患者id
        self.nodule_no = nodule_no # 结节id 
        self.slice_thickness = slice_thickness # 层厚
        self.image_list = []
        self.mask_list = []

    def __repr__(self):
        return f"<Nodule-{self.patient_id}-{self.nodule_no}>"

    def add_image(self, image_path, mask_path):
        self.image_list.append(image_path)
        self.mask_list.append(mask_path)
    
    @classmethod
    def get_union_bbox(cls, bboxs):
        """
        计算多个锚框的最小并集锚框
        :param bboxs: 包含多个锚框的列表，每个锚框由左上角和宽高表示 [(x, y, w, h), ...]
        :return: 最小并集锚框的坐标 (x_min, y_min, x_max, y_max)
        """
        # 初始化最小左上角和最大右下角
        x_min = float('inf')
        y_min = float('inf')
        x_max = float('-inf')
        y_max = float('-inf')
        
        for (x, y, w, h, _) in bboxs:
            # 计算右下角坐标
            x2 = x + w
            y2 = y + h
            
            # 更新最小左上角和最大右下角
            x_min = min(x_min, x)
            y_min = min(y_min, y)
            x_max = max(x_max, x2)
            y_max = max(y_max, y2)
        return (x_min, y_min, x_max, y_max)
    
    @classmethod
    def adjust_layer_thickness(cls, stacked_data, original_thickness, target_thickness):
        """
        调整 3D 数据中每一层的厚度为目标厚度，通过插值方法。
        
        :param stacked_data: 原始的 3D 数据，形状为 (num_layers, height, width)
        :param original_thickness: 每一层的原始厚度（单位：mm），假设所有层厚度相同
        :param target_thickness: 目标层厚度（单位：mm）
        
        :return: 调整后的 3D 数据，层厚统一为目标层厚度
        """
        # 计算缩放因子
        scaling_factor = original_thickness / target_thickness
        print(f"Scaling factor for Z-axis: {scaling_factor}")

        # 获取原始 3D 数据的形状 (num_layers, height, width)
        num_layers, height, width = stacked_data.shape

        # 对整个 3D 数据在 Z 轴方向进行插值
        rescaled_data = scipy.ndimage.zoom(stacked_data, (scaling_factor, 1, 1), order=1)

        return rescaled_data

    def construct_3d(self, adjust_thickness=False, target_thickness=1.25):
        """
        根据image和mask 还有层厚构建3d对象
        """
        # 统计阶段
        # 1、获取所有mask的bboxs和images
        masks = [np.load(mask) for mask in self.mask_list]
        bboxs = [mask_find_bboxs(mask)[0] for mask in masks]
        images = [np.load(img) for img in self.image_list]

        # 2、根据bbox获取相应的图片截断部分
        union_bbox = self.get_union_bbox(bboxs)
        x_min, y_min, x_max, y_max = union_bbox
        images = [img[y_min:y_max, x_min:x_max] for img in images]
        stacked_data = np.stack(images, axis=0)
        if adjust_thickness:
            return self.adjust_layer_thickness(stacked_data, self.slice_thickness, target_thickness)
        else:
            return stacked_data
    
    @property
    def z_length(self):
        return self.slice_thickness*len(self.mask_list)

In [3]:
mask_list = ['E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice000.npy', 'E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice001.npy', 'E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice002.npy', 'E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice003.npy', 'E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice004.npy', 'E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice005.npy', 'E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice006.npy', 'E:/Processed-LIDC-full-lung/Mask/LIDC-IDRI-0001/0001_MA000_slice007.npy']
image_list = ['E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice000.npy', 'E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice001.npy', 'E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice002.npy', 'E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice003.npy', 'E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice004.npy', 'E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice005.npy', 'E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice006.npy', 'E:/Processed-LIDC-full-lung/Image/LIDC-IDRI-0001/0001_NI000_slice007.npy']

In [4]:
nodule = Nodule(patient_id=1, nodule_no=0, slice_thickness=2.5)
nodule.mask_list = mask_list
nodule.image_list = image_list

In [5]:
obj_3d = nodule.construct_3d(adjust_thickness=True)
obj_3d.shape

Scaling factor for Z-axis: 2.0


(16, 45, 37)

In [7]:
import numpy as np
import vtk

# 获取数据的维度
dims = obj.shape

# 创建一个 vtkImageData 对象来存储 3D 数据
image_data = vtk.vtkImageData()
image_data.SetDimensions(dims[2], dims[1], dims[0])  # x, y, z

# 将 numpy 数据转换为 vtk 数组
flat_data = obj.flatten()  # 将数据展平

# 使用 vtk 库的 vtkFloatArray 来存储数据
vtk_data = vtk.vtkFloatArray()
vtk_data.SetArray(flat_data, len(flat_data), 1)  # 数组数据，长度，是否是固定长度

# 设置 image_data 的点数据
image_data.GetPointData().SetScalars(vtk_data)

# 创建一个渲染器、渲染窗口和交互式渲染窗口
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# 使用 vtk 抽象的可视化方法来显示数据
volume_mapper = vtk.vtkGPUVolumeRayCastMapper()
volume_mapper.SetInputData(image_data)

# 创建一个体积属性对象，用于设置颜色和透明度
volume_property = vtk.vtkVolumeProperty()
volume_property.SetColor(vtk.vtkColorTransferFunction())
volume_property.SetScalarOpacity(vtk.vtkPiecewiseFunction())

# 创建一个体积对象
volume = vtk.vtkVolume()
volume.SetMapper(volume_mapper)
volume.SetProperty(volume_property)

# 将体积对象添加到渲染器
renderer.AddVolume(volume)

# 设置背景颜色
renderer.SetBackground(0.1, 0.1, 0.1)  # 背景颜色

# 添加坐标轴
axes = vtk.vtkAxesActor()

# 创建坐标轴标尺
axes.SetTotalLength(10, 10, 10)  # 设定坐标轴的总长度，单位为坐标轴的缩放比例
axes.GetXAxisCaptionActor2D().GetCaptionTextProperty().SetColor(1.0, 0.0, 0.0)  # 设置x轴标题颜色为红色
axes.GetYAxisCaptionActor2D().GetCaptionTextProperty().SetColor(0.0, 1.0, 0.0)  # 设置y轴标题颜色为绿色
axes.GetZAxisCaptionActor2D().GetCaptionTextProperty().SetColor(0.0, 0.0, 1.0)  # 设置z轴标题颜色为蓝色

# 将坐标轴添加到渲染器
renderer.AddActor(axes)

# 创建一个 3D 标记，显示坐标轴的标签
renderer.SetBackground(0.1, 0.1, 0.1)  # 背景颜色

# 启动渲染
render_window.Render()
render_window_interactor.Start()